In [ ]:
# ======================================================
# 📓 CBM_persona.ipynb
# - CatBoost (.cbm) 기반 이직 성공 확률 예측
# - 페르소나 테스트
# - 규칙 기반 Recourse (DiCE 미사용)
# ======================================================


# ======================================================
# 🟦 [셀 1] 라이브러리
# ======================================================
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier

# ======================================================
# 🟦 [셀 2] CBM 모델 로드
# ======================================================

MODEL_PATH = "../models/RYUI/cat_model.cbm"

cbm_model = CatBoostClassifier()
cbm_model.load_model(MODEL_PATH)

# 학습 당시 feature 순서 (중요)
EXPECTED_COLS = cbm_model.feature_names_

print("✅ CBM model loaded")
print("Feature count:", len(EXPECTED_COLS))

# ======================================================
# 🟦 [셀 3] 공통 예측 함수
# ======================================================

def _align_cols(X: pd.DataFrame) -> pd.DataFrame:
    """
    CatBoost 학습 당시 컬럼 순서로 정렬
    """
    return X.reindex(columns=EXPECTED_COLS)


def predict_cbm_probability(X: pd.DataFrame) -> float:
    """
    이직 성공 확률 반환
    """
    X = _align_cols(X)
    return cbm_model.predict_proba(X)[:, 1][0]


# ======================================================
# 🟦 [셀 4] Actionable Feature 정의
# ======================================================

ACTIONABLE_FEATURES = [
    "직업교육및직장체험_취업관련시험준비여부",
    "직업교육및직장체험_취업관련시험준비분야코드",
    "직업교육및직장체험_직장체험주요형태코드",
    "직업교육및직장체험_재학휴학중직장체험여부",
    "직업교육및직장체험_재학휴학중직장체험기간코드",
    "구직사항_구직활동기간",
]

# ======================================================
# 🟦 [셀 5] 규칙 기반 Recourse 시뮬레이션
# ======================================================

def simulate_recourse(X: pd.DataFrame, min_gain=0.01):
    """
    각 변수 +1 변화 시 확률 증가 여부 확인
    """
    X = _align_cols(X)
    base_prob = predict_cbm_probability(X)

    results = []

    for f in ACTIONABLE_FEATURES:
        if f not in X.columns:
            continue

        X2 = X.copy()

        # 수치형만 변화
        if np.issubdtype(X2[f].dtype, np.number):
            X2[f] = X2[f] + 1
        else:
            continue

        new_prob = predict_cbm_probability(X2)
        delta = new_prob - base_prob

        if delta >= min_gain:
            results.append({
                "feature": f,
                "prob_before": round(base_prob, 4),
                "prob_after": round(new_prob, 4),
                "gain": round(delta, 4)
            })

    return sorted(results, key=lambda x: x["gain"], reverse=True)

# ======================================================
# 🟦 [셀 6] 페르소나 입력 (자유 수정)
# ======================================================

persona = {
    "성별코드": 2,
    "만연령": 30,
    "연령계층코드": 3,
    "혼인상태코드": 1,
    "가구주관계코드": 3,
    "교육정도_학력구분코드": 5,
    "교육정도_계열코드": 20,
    "가장최근학교_년제수": 4,
    "졸업연도": 2019,
    "가장최근학교_휴학경험유무": 1,
    "가장최근학교_휴학기간년수": 1,
    "직업교육및직장체험_직업교육수혜구분코드": 1,
    "직업교육및직장체험_취업관련시험준비여부": 0,
    "직업교육및직장체험_취업관련시험준비분야코드": 0,
    "직업교육및직장체험_직장체험주요형태코드": 0,
    "직업교육및직장체험_재학휴학중직장체험여부": 0,
    "직업교육및직장체험_재학휴학중직장체험기간코드": 0,
    "구직사항_구직활동기간": 6,
    "구직사항_주요구직방법1코드": 3,
    "구직사항_주요구직경로1코드": 2
}

X_persona = pd.DataFrame([persona])

# ======================================================
# 🟦 [셀 7] 페르소나 예측 결과
# ======================================================

cbm_prob = predict_cbm_probability(X_persona)
recourse = simulate_recourse(X_persona)

print("="*60)
print("📌 CBM 페르소나 테스트 결과")
print("이직 성공 확률:", round(cbm_prob, 4))
print("="*60)

print("\n📌 Recourse (확률 증가 순)")
for r in recourse:
    print(r)


✅ CBM model loaded: ../models/RYUI/cat_model.cbm


CatBoostError: catboost/libs/data/model_dataset_compatibility.cpp:81: At position 0 should be feature with name 만연령 (found 성별코드).